# Uploading a Qiskit runtime program

<div class="alert alert-block alert-info">
<b>Note:</b> Access to the Qiskit Runtime service may not mean you have access to upload a runtime program, but check back, as we'll be releasing it publicly soon!
</div>

Here we provide an overview on how to construct and upload a runtime program. A runtime program is a piece of Python code that lives in the cloud and can be invoked by other authorized users. Currently all runtime programs are **public**, meaning they are available to be used by authorized users as soon as they are uploaded.

## Constructing a runtime program

Below is a template of a runtime program. You can find the template file in the 
[`qiskit-ibmq-provider`](https://github.com/Qiskit/qiskit-ibmq-provider/blob/master/qiskit/providers/ibmq/runtime/program/program_template.py) repository.

In [1]:
import sys
import json

from qiskit.providers.ibmq.runtime import UserMessenger, ProgramBackend


def program(backend: ProgramBackend, user_messenger: UserMessenger, **kwargs):
    """Function that does classical-quantum calculation."""
    # UserMessenger can be used to publish interim results.
    user_messenger.publish("This is an interim result.")
    return "final result"


def main(backend: ProgramBackend, user_messenger: UserMessenger, **kwargs):
    """This is the main entry point of a runtime program.

    The name of this method must not change. It also must have ``backend``
    and ``user_messenger`` as the first two positional arguments.

    Args:
        backend: Backend for the circuits to run on.
        user_messenger: Used to communicate with the program user.
        kwargs: User inputs.
    """
    # Massage the input if necessary.
    result = program(backend, user_messenger, **kwargs)
    # UserMessenger can be used to publish final results.
    user_messenger.publish(result, final=True)  

Each runtime program must have a `main()` function, which serves as the entry point to the program. This function must have `backend` and `user_messenger` as the first two positional arguments:

- `backend` is an instance of [`ProgramBackend`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.ProgramBackend.html#qiskit.providers.ibmq.runtime.ProgramBackend) and has a `run()` method that can be used to submit circuits.
- `user_messenger` is an instance of [`UserMessenger`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.UserMessenger.html#qiskit.providers.ibmq.runtime.UserMessenger) and has a `publish()` method that can be used to send interim and final results to the program user. This method takes a parameter `final` that indicates whether it's a final result. Currently only final results are stored after a program execution finishes.

There are several runtime programs in the `programs` directory in this repository. `sample_program.py` is one of them. It is a sample runtime program that submits random circuits for user-specified iterations:

In [2]:
"""A sample runtime program that submits random circuits for user-specified iterations."""

import random

from qiskit import transpile
from qiskit.circuit.random import random_circuit


def prepare_circuits(backend):
    """Generate a random circuit.

    Args:
        backend: Backend used for transpilation.

    Returns:
        Generated circuit.
    """
    circuit = random_circuit(num_qubits=5, depth=4, measure=True,
                             seed=random.randint(0, 1000))
    return transpile(circuit, backend)


def main(backend, user_messenger, **kwargs):
    """Main entry point of the program.

    Args:
        backend: Backend to submit the circuits to.
        user_messenger: Used to communicate with the program consumer.
        kwargs: User inputs.
    """
    iterations = kwargs.pop('iterations', 5)
    for it in range(iterations):
        qc = prepare_circuits(backend)
        result = backend.run(qc).result()
        user_messenger.publish({"iteration": it, "counts": result.get_counts()})

    user_messenger.publish("All done!", final=True)


## Data serialization

Runtime programs live in the cloud, and JSON is the standard way of passing data to cloud services. Therefore, when a user invokes a runtime program, the input parameters must first be serialized into the JSON format before passed to the server and then deserialized once received by the server. This serialization and deserialization is done using the [`RuntimeEncoder`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.RuntimeEncoder.html#qiskit.providers.ibmq.runtime.RuntimeEncoder) and [`RuntimeDecoder`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.RuntimeDecoder.html#qiskit.providers.ibmq.runtime.RuntimeDecoder) classes, respectively.

Similarly, results passed back by a runtime program need to be serialized. You can choose to use the default `RuntimeEncoder`, or pass your own encoder to `user_messenger.publish()`. Keep in mind that if you choose to use your own encoder, you must document the corresponding decoder needed for the users of your program to deserialize the results.

If you want to pass custom Python classes, the `RuntimeEncoder` and `RuntimeDecoder` methods have some support for that. In your custom class, you can define a `to_json()` method that returns a JSON string representation of the object, and a `from_json()` class method that accepts a JSON string and returns the corresponding object. When `RuntimeEncoder` serializes a Python object, it checks whether the object has a `to_json()` method. If so, it uses the method for serialization. `RuntimeDecoder`, however, does _not_ invoke `from_json()` to convert the data back because it doesn't know how to import your custom class. You can, however, create your own decoder.

You decoder should inherit the [`ResultDecoder`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.ResultDecoder.html#qiskit.providers.ibmq.runtime.ResultDecoder) class and overwrites its `decode()` method. This `decode()` method is called to deserialize job results. Your subclass can call the parent's ``decode()`` method to handle deserialization of other result data, and the subclass can handle just the custom class. 

Here is an example of using a custom class `MyCustomClass`:

In [3]:
import json

class MyCustomClass:
    
    def __init__(self, foo, bar):
        self._foo = foo
        self._bar = bar
    
    def to_json(self):
        return json.dumps({"foo": self._foo, "bar": self._bar})
    
    @classmethod
    def from_json(cls, json_str):
        return cls(**json.loads(json_str))

If an instance of `MyCustomClass` is used as the input, `RuntimeEncoder` will invoke its `to_json()` method to serialize the data. It also adds a `__type__` field to differentiate it from other types of data:

In [4]:
from qiskit.providers.ibmq.runtime import RuntimeEncoder

program_inputs = {
    'my_obj': MyCustomClass("my foo", "my bar")
}

# The following comments show how it's normally used by the program user:

# options = {'backend_name': backend.name()}
# job = provider.runtime.run(program_id="my-program",
#                            options=options,
#                            inputs=program_inputs
#                           )

# This shows what the encoded value looks like:
serialized = json.dumps(program_inputs, cls=RuntimeEncoder)
print(f"This is what gets passed to the program:\n {serialized}")

This is what gets passed to the program:
 {"my_obj": {"__type__": "to_json", "__value__": "{\"foo\": \"my foo\", \"bar\": \"my bar\"}"}}


Your program can then use the `from_json()` method to restore the object:

In [5]:
def main(backend, user_messenger, **kwargs):
    """Main entry point of the program."""
    my_obj_str = kwargs.pop('my_obj')
    my_obj = MyCustomClass.from_json(my_obj_str)

Similarly, if an instance of `MyCustomClass` is passed back as a program result:

In [6]:
def main(backend, user_messenger, **kwargs):
    """Main entry point of the program."""
    user_messenger.publish({"my_obj": MyCustomClass("this foo", "that bar")}, final=True)

Then you can define a custom `ResultDecoder.decode()` to reconstruct the object:

In [7]:
from qiskit.providers.ibmq.runtime import ResultDecoder

class MyResultDecoder(ResultDecoder):

    @classmethod
    def decode(cls, data):
        decoded = super().decode(data)  # Call parent method to handle other data.
        decoded["my_obj"] = MyCustomClass.from_json(decoded["my_obj"])
        return decoded

Users of your program can then use `MyResultDecoder` to decode the results:

In [8]:
# The following comments shows how it's normally used by the program user:

# job = provider.runtime.run(program_id="my-program",
#                            options=options,
#                            inputs=program_inputs,
#                            result_decoder=MyResultDecoder
#                           )

# This shows what the decoded value looks like:
MyResultDecoder.decode(serialized)

{'my_obj': <__main__.MyCustomClass at 0x133192a60>}

## Testing your runtime program

You can test your runtime program using a local simulator or a real backend before uploading it. Simply import and invoke the `main()` function of your program and pass the following parameters:

- the `backend` instance you want to use
- a new `UserMessenger` instance.
- program input parameters that are serialized and then deserialized using the correct encoder and decoder. While this may seem redundant, it is to ensure input parameters can be passed to your program properly once it's uploaded to the cloud.


The following example tests the `sample-program` program we saw earlier. It uses the `qasm_simulator` from Qiskit Aer as the test backend. It serializes and unserializes input data using `RuntimeEncoder` and `RuntimeDecoder`, which are the default en/decoders used by runtime.

In [9]:
import sys
sys.path.insert(0, '..') # Add qiskit_runtime directory to the path

from qiskit_runtime.sample_program import sample_program
from qiskit import Aer
from qiskit.providers.ibmq.runtime.utils import RuntimeEncoder, RuntimeDecoder
from qiskit.providers.ibmq.runtime import UserMessenger

inputs = {"iterations": 3}

backend = Aer.get_backend('qasm_simulator')
user_messenger = UserMessenger()
serialized_inputs = json.dumps(inputs, cls=RuntimeEncoder)
unserialized_inputs = json.loads(serialized_inputs, cls=RuntimeDecoder)

sample_program.main(backend, user_messenger, **unserialized_inputs)

{"iteration": 0, "counts": {"10111": 269, "00110": 240, "00100": 242, "10101": 273}}
{"iteration": 1, "counts": {"10000": 506, "00000": 518}}
{"iteration": 2, "counts": {"00001": 408, "00011": 616}}
"All done!"


## Defining program metadata

Program metadata is data that helps users to understand how to invoke your program. It includes:

- `name`: Name of the program. This must be unique. 
- `maximum execution time`: Maximum amount of time, in seconds, a program can run before being forcibly terminated.
- `description`: Describes the program.
- `version`: Program version.
- `backend_requirements`: Describes the backend attributes needed to run the program.
- `parameters`: Describes the program input parameters
- `return_values`: Describes the return values
- `interim_results`: Describes the interim results

When uploading a program, you must specify at least `name`, `maximum execution time`, and `description`. It is strongly encouraged to also specify `parameters`, `return values`, and `interim results` if the program has them. You can specify the metadata fields individually, or use a JSON file.

Below shows the metadata JSON file of the `sample-program` program as an example:

In [10]:
import os

sample_program_json = os.path.join(os.getcwd(), "../qiskit_runtime/sample_program/sample_program.json")

with open(sample_program_json, 'r') as file:
    data = file.read()

print(data)

{
  "name": "sample-program",
  "description": "A sample runtime program.",
  "max_execution_time": 300,
  "version": "1.0",
  "backend_requirements": {"min_num_qubits":  5},
  "parameters": [
    {"name": "iterations", "description": "Number of iterations to run. Each iteration generates and runs a random circuit.", "type": "int", "required": true}
  ],
  "return_values": [
    {"name": "-", "description": "A string that says 'All done!'.", "type": "string"}
  ],
  "interim_results": [
    {"name": "iteration", "description": "Iteration number.", "type": "int"},
    {"name": "counts", "description": "Histogram data of the circuit result.", "type": "dict"}
  ]
}



Note that the return value has `"name": "-"` because there is only 1 return value. If the program has multiple return values, you should specify a meaningful name for each one.

## Uploading a program

You can use the [`IBMRuntimeService.upload_program()`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.IBMRuntimeService.html#qiskit.providers.ibmq.runtime.IBMRuntimeService.upload_program) method to upload your program. In the example below, the program data lives in the file `sample_program.py`, and its metadata, as described above, is in `sample_program.json`. 

In [11]:
import os
from qiskit import IBMQ

IBMQ.load_account()
provider = IBMQ.get_provider(project='qiskit-runtime')  # Substitute with your provider.

sample_program_data = os.path.join(os.getcwd(), "../qiskit_runtime/sample_program/sample_program.py")
sample_program_json = os.path.join(os.getcwd(), "../qiskit_runtime/sample_program/sample_program.json")
    
program_id = provider.runtime.upload_program(
    data=sample_program_data,
    metadata=sample_program_json
)
print(program_id)

sample-program


`upload_program()` returns a program ID, which uniquely identifies the program. It is derived from the program name but is not guaranteed to be the same. Program ID is needed for users to invoke the program.

## Deleting a program

You can use the [`IBMRuntimeService.delete_program()`](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.runtime.IBMRuntimeService.html#qiskit.providers.ibmq.runtime.IBMRuntimeService.delete_program) method to delete a program. Only the original author of a program can delete it. 


In [12]:
import qiskit.tools.jupyter
%qiskit_version_table